In [1]:
import numpy as np
from scipy.spatial.transform import Rotation

def transform_euler_angles(quaternion):
    """
    使用四元数转换欧拉角，并将其转换为特定形式的欧拉角。
    
    Args:
        quaternion (list): 长度为4的列表，表示四元数。
    
    Returns:
        list: 长度为3的列表，表示经过转换的欧拉角。
    """
    # 使用四元数转换欧拉角
    rotation = Rotation.from_quat(quaternion)
    euler_angles = rotation.as_euler("zyx", degrees=True)
    
    # 将欧拉角转换为特定形式
    # 仅使用 transform 转化，得到的数据不正确，需要进一步转化
    # 第一个角度不需要变化，第二个角度前面加负数，第三个是和180互补加负数
    transformed_angles = np.round([euler_angles[0], -euler_angles[1], -(180 - euler_angles[2])], 6)
    
    return transformed_angles

In [ ]:
## 探究阿鲁比的传感器是否有做算法优化

In [1]:

import pandas as pd
file =  "/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230424鞋垫空放自重漂移测试/RawData/hwshuai_20230424-丁冠宏-步频100-4来回-鞋垫2-1-41AB-V4.02-001_202304241804055775.csv"
df = pd.read_csv(file)
df.columns


Index(['part', 'timestamp', 'gx', 'gy', 'gz', 'ax', 'ay', 'az', 'qw', 'qx',
       'qy', 'qz', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
       'p11', 'p12', 'p1-c', 'p2-c', 'p3-c', 'p4-c', 'p5-c', 'p6-c', 'p7-c',
       'p8-c', 'p9-c', 'p10-c', 'p11-c', 'p12-c'],
      dtype='object')

In [5]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

def transform_euler_angles(quaternion):
    """
    使用四元数转换欧拉角，并将其转换为特定形式的欧拉角。
    使用阿鲁比小蓝块 IMU 数据进行验证，没有进行鞋垫姿态传感器的验证
    
    Args:
        quaternion (list): 长度为4的Series，表示四元数。
    
    Returns:
        pd.Series: 长度为3的Series，表示经过转换的欧拉角。
    """
    # 使用四元数转换欧拉角
    rotation = Rotation.from_quat(quaternion)
    euler_angles = rotation.as_euler("zyx", degrees=True)
    
    # 将欧拉角转换为特定形式
    # 仅使用 transform 转化，得到的数据不正确，需要进一步转化
    # 第一个角度不需要变化，第二个角度前面加负数，第三个是和180互补加负数
    # 欧拉角的角度范围应该是 -180～180，但是现在数据范围大于 180
    if (euler_angles[2] < 0 or euler_angles[2] == 0).all():
        transformed_angles = np.round([euler_angles[0], -euler_angles[1], 180 + euler_angles[2]], 6)
    else:
        transformed_angles = np.round([euler_angles[0], -euler_angles[1], -(180 - euler_angles[2])], 6)

    # transformed_angles = np.round([euler_angles[0], euler_angles[1], euler_angles[2]], 6)
    
    return pd.Series(transformed_angles)
eulers = df[['qw', 'qx','qy', 'qz']].apply(transform_euler_angles,axis=1)
eulers.columns = ["x","y","z"]

In [20]:
# 将角速度进行积分，得到角度的变化量
dt = 1/50 #采样周期
dtheta = df['ax'].apply(lambda x: x*dt, axis=1)

# 将角度的变化量进行累积，得到角度的时间序列
theta = dtheta.cumsum()


KeyError: 'ax'

In [15]:
import pandas as pd
import numpy as np

# 创建一个包含角速度的 DataFrame
df = pd.DataFrame({'wx': [0, 1, 2, 3], 'wy': [0, 0, 1, 1], 'wz': [0, 0, 0, 1]}, index=[0, 1, 2, 3])

# 将角速度进行积分，得到角度的变化量
dt = 1  # 采样周期
dtheta = df.apply(lambda x: x * dt, axis=0)
print(dtheta)

# 将角度的变化量进行累积，得到角度的时间序列
theta = dtheta.cumsum()
print(theta)

   wx  wy  wz
0   0   0   0
1   1   0   0
2   2   1   0
3   3   1   1
   wx  wy  wz
0   0   0   0
1   1   0   0
2   3   1   0
3   6   2   1
